In [1]:
# Load data from training data directory, convert from NPZ to tensor, and save to a dict
# containing lift name and tensor data

import os
import torch
import numpy as np
import functions

DEADLIFT_TENSORS = {}
BENCH_TENSORS = {}
SQUAT_TENSORS = {}

# Setup paths
script_dir = os.path.dirname(os.path.abspath("model_train.ipynb"))
data_dir = os.path.normpath(os.path.join(script_dir, '..', 'training data'))

# Step 1: Get all subfolder names in data_dir
subfolders = []
for f in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, f)):
        subfolders.append(f)


#  Squat Files
#  Bench Files
#  Deadlift Files
for subfolder in subfolders:
    subfolder_path = os.path.join(data_dir, subfolder)
    files = []

    # Isolate individual lift files in each of the subfolders and add them to files list
    for f in os.listdir(subfolder_path):
        if os.path.isfile(os.path.join(subfolder_path, f)):
            files.append(f)

    # Iterate through files in each subfolder
    for file in files:
        data = np.load(os.path.join(subfolder_path, file))

        # convert each landmark/angle to a tensor and add it to the tensor dict (which is
        # specific to the current lift only
        tensor_dict = {}
        for key in data.files:
            lift_name_without_video_type = " ".join(key.split('.')[:1] + key.split("_")[-2:])
            tensor_dict[lift_name_without_video_type] = torch.tensor(data[key])
        if file.split('.')[0].startswith('bench'):
            BENCH_TENSORS[file.split('.')[0]] = tensor_dict
        if file.split('.')[0].startswith('deadlift'):
            DEADLIFT_TENSORS[file.split('.')[0]] = tensor_dict
        if file.split('.')[0].startswith('squat'):
            SQUAT_TENSORS[file.split('.')[0]] = tensor_dict

In [2]:
BENCH_TENSORS['bench 2 good lift data'].keys()

dict_keys(['bench 2 good landmark 0', 'bench 2 good landmark 1', 'bench 2 good landmark 2', 'bench 2 good landmark 3', 'bench 2 good landmark 4', 'bench 2 good landmark 5', 'bench 2 good landmark 6', 'bench 2 good landmark 7', 'bench 2 good landmark 8', 'bench 2 good landmark 9', 'bench 2 good landmark 10', 'bench 2 good landmark 11', 'bench 2 good landmark 12', 'bench 2 good landmark 13', 'bench 2 good landmark 14', 'bench 2 good landmark 15', 'bench 2 good landmark 16', 'bench 2 good landmark 17', 'bench 2 good landmark 18', 'bench 2 good landmark 19', 'bench 2 good landmark 20', 'bench 2 good landmark 21', 'bench 2 good landmark 22', 'bench 2 good landmark 23', 'bench 2 good landmark 24', 'bench 2 good landmark 25', 'bench 2 good landmark 26', 'bench 2 good landmark 27', 'bench 2 good landmark 28', 'bench 2 good landmark 29', 'bench 2 good landmark 30', 'bench 2 good landmark 31', 'bench 2 good landmark 32', 'bench 2 good left elbow', 'bench 2 good right elbow', 'bench 2 good left k

In [3]:
for k,v in BENCH_TENSORS.items():
    print(k)
    for key in v.keys():

        try:
            print(f"---{key} contains a {type(v[key])} containing Z coords for {functions.landmark_indeces_to_labels[int(key.split('_')[-1])]} that contains {len(v[key])} points")
        except:
            print(f"---{key} contains a {type(v[key])} containing joint angles for {' '.join(key.split('_')[-2:])} that contains ({len(v[key])} angles)")

bench 2 good lift data
---bench 2 good landmark 0 contains a <class 'torch.Tensor'> containing joint angles for bench 2 good landmark 0 that contains (3 angles)
---bench 2 good landmark 1 contains a <class 'torch.Tensor'> containing joint angles for bench 2 good landmark 1 that contains (3 angles)
---bench 2 good landmark 2 contains a <class 'torch.Tensor'> containing joint angles for bench 2 good landmark 2 that contains (3 angles)
---bench 2 good landmark 3 contains a <class 'torch.Tensor'> containing joint angles for bench 2 good landmark 3 that contains (3 angles)
---bench 2 good landmark 4 contains a <class 'torch.Tensor'> containing joint angles for bench 2 good landmark 4 that contains (3 angles)
---bench 2 good landmark 5 contains a <class 'torch.Tensor'> containing joint angles for bench 2 good landmark 5 that contains (3 angles)
---bench 2 good landmark 6 contains a <class 'torch.Tensor'> containing joint angles for bench 2 good landmark 6 that contains (3 angles)
---bench 2 

In [4]:
# for k, v in BENCH_TENSORS.items():
#     for lift, coordinate in v.items():
#         print(lift, "coords",coordinate[2])

for key, value in DEADLIFT_TENSORS.items():
    print(key)
    # for landmark, coordinate in BENCH_TENSORS[key].items():
    #     return (landmark, coordinate[2])

deadlift 1 good lift data
deadlift 2 good_rotate_15 lift data
deadlift 3 bad lift data
deadlift 4 bad lift data
deadlift 6 good lift data


In [6]:
DEADLIFT_TENSORS['deadlift 1 good lift data'].keys()

dict_keys(['deadlift 1 good landmark 0', 'deadlift 1 good landmark 1', 'deadlift 1 good landmark 2', 'deadlift 1 good landmark 3', 'deadlift 1 good landmark 4', 'deadlift 1 good landmark 5', 'deadlift 1 good landmark 6', 'deadlift 1 good landmark 7', 'deadlift 1 good landmark 8', 'deadlift 1 good landmark 9', 'deadlift 1 good landmark 10', 'deadlift 1 good landmark 11', 'deadlift 1 good landmark 12', 'deadlift 1 good landmark 13', 'deadlift 1 good landmark 14', 'deadlift 1 good landmark 15', 'deadlift 1 good landmark 16', 'deadlift 1 good landmark 17', 'deadlift 1 good landmark 18', 'deadlift 1 good landmark 19', 'deadlift 1 good landmark 20', 'deadlift 1 good landmark 21', 'deadlift 1 good landmark 22', 'deadlift 1 good landmark 23', 'deadlift 1 good landmark 24', 'deadlift 1 good landmark 25', 'deadlift 1 good landmark 26', 'deadlift 1 good landmark 27', 'deadlift 1 good landmark 28', 'deadlift 1 good landmark 29', 'deadlift 1 good landmark 30', 'deadlift 1 good landmark 31', 'deadli

In [7]:
all_deadlift_joint_angles = functions.joint_angles_to_list(DEADLIFT_TENSORS)
all_bench_joint_angles = functions.joint_angles_to_list(BENCH_TENSORS)
all_squat_joint_angles = functions.joint_angles_to_list(SQUAT_TENSORS)

# Example of one output for one lift (in this case, deadlift 1 good is the first to be
# processed, so it is at index 0 of our array. We can now iterate over this array with our
# model to learn patterns
all_deadlift_joint_angles[0]

{'name': 'deadlift',
 'viewing from': 'right',
 'label': 'good',
 'angles': {'left elbow': tensor([ 77,  70,  68,  65,  62,  58,  54,  53,  51,  49,  48,  46,  44,  42,
           42,  41,  42, 126, 178, 178, 178, 178, 177, 177, 177, 177, 177, 176,
          176, 176, 176, 176, 176, 176, 176, 176, 176, 176, 177, 177, 177, 178,
          178, 178, 178, 178, 178, 178, 178, 178, 178, 178, 177, 177, 177, 177,
          176, 176, 176, 176, 175, 175, 175, 174, 174, 174, 174, 173, 173, 173,
          173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 174,
          174, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173, 173,
          174, 174, 174, 174, 174, 175, 175, 175, 175, 175, 175, 175, 175, 175,
          175, 175, 175, 174, 174, 174, 174, 173, 173, 173, 172, 172, 172, 172,
          172, 172, 172, 173, 173, 173, 173, 174, 174, 174, 174, 174, 174, 174,
          174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174, 174,
          174, 174, 173, 173, 1